# Convert Wingtra flight plan to KML

In [123]:
# Author: Farid Javadnejad
# Date: 2025-02-18
#
# DESCRIPTION:
# This script converts a Wingtra UAS flight plan from a .flightplan file (i.e., JSON) to a KML file, 
# extracting the survey area and adding a Placemark for the takeoff/landing location.
#
# DISCLAIMER:
# This script was developed with the assistance of AI tools for debugging, reviewing, and testing.
# ---------------------------------------------------------------------------------------------------

In [124]:
#pip install simplekml fastkml lxml
import json
import os
import simplekml
import random

In [129]:
# Colors 
Blue = (0,170,255) 
Green = (0,255,0) 
Orange = (255,170,0) 
Magenta = (255,0,255)

colors = [Blue, Green, Orange, Magenta]

#Global color defintion
global FILL_COLOR


In [126]:
def read_flightplan(file_path):
    with open(file_path, "r") as file:
        return json.load(file)

def extract_survey_area(flightplan):
    for item in flightplan.get("flightPlan", {}).get("items", []):
        if item.get("complexItemType") == "area" and "polygon" in item:
            return item["polygon"]
    return None

def extract_corridors(flightplan):
    corridors = []
    for item in flightplan.get("flightPlan", {}).get("items", []):
        if item.get("complexItemType") == "corridor" and "corridor" in item:
            polyline = item["corridor"].get("polyline", [])
            if polyline:
                corridors.append(polyline)
    return corridors

def convert_to_kml_coordinates(coords):
    try:
        return [(point[1], point[0]) for point in coords]
    except IndexError:
        print("Invalid coordinate format.")
        return None

import simplekml
import os

def create_kml_file(kml_coordinates, corridors, takeoff_landing, file_path):
    kml = simplekml.Kml()

    # Add area polygon
    if kml_coordinates:

        pol = kml.newpolygon(name="Flight Area", outerboundaryis=kml_coordinates)
        pol.style.polystyle.color = simplekml.Color.rgb(FILL_COLOR[0], FILL_COLOR[1], FILL_COLOR[2], 128) #50% opacity (128/255)
        pol.style.linestyle.color = simplekml.Color.rgb(255, 255, 255)
        pol.style.linestyle.width = 1.0
    
    # Add corridors as polylines
    for i, corridor in enumerate(corridors):
        kml_coords = convert_to_kml_coordinates(corridor)
        #randoly choose a color
        if kml_coords:
            line = kml.newlinestring(name=f"Corridor {i+1}", coords=kml_coords)
            line.style.linestyle.color = simplekml.Color.rgb(FILL_COLOR[0], FILL_COLOR[1], FILL_COLOR[2], 128) #50% opacity (128/255)
            line.style.linestyle.width = 100
    
    # Add takeoff/landing point with custom icon
    if takeoff_landing:
        point = kml.newpoint(name="Takeoff/Landing", coords=[(takeoff_landing[1], takeoff_landing[0], takeoff_landing[2])])
        point.style.iconstyle.icon.href = "https://earth.google.com/images/kml-icons/track-directional/track-0.png"
        point.style.iconstyle.scale = 1.1  # Icon size
        point.style.labelstyle.scale = 0.7  # Label size
    
    kml_filename = os.path.splitext(file_path)[0] + ".kml"
    kml.save(kml_filename)
    print(f"KML file saved: {kml_filename}")


def convert_flightplan_to_kml(file_path):
    flightplan = read_flightplan(file_path)
    survey_area = extract_survey_area(flightplan)
    corridors = extract_corridors(flightplan)

    kml_coordinates = convert_to_kml_coordinates(survey_area) if survey_area else None
    takeoff_landing = flightplan.get("flightPlan", {}).get("items", [])[0].get("coordinate", [])
    create_kml_file(kml_coordinates, corridors, takeoff_landing, file_path)

In [ ]:
# Define the folder containing the target files
read_folder = "C:/Farid/Projects/20250310 - MAT PN1 TO1/UAS/00_PLANNING/Wingtra Flight Plan"

# Define extention
read_file_extension = ".flightplan"

# Color: Initialize an index variable to track the current color
index = 0

# Loop through all target files in the read_folder
for read_file in os.listdir(read_folder):
    
    ## COLOR
    # Pick the color based on the current index, then increment the index
    FILL_COLOR = colors[index]
    # Increment the index and cycle back to the beginning using modulo
    index = (index + 1) % len(colors)

    # READ FILE AND CREATE KML
    if read_file.endswith(read_file_extension):
        file_path = os.path.join(read_folder, read_file)
        convert_flightplan_to_kml(file_path)


KML file saved: C:/Farid/Projects/20250310 - MAT PN1 TO1/UAS/00_PLANNING/Wingtra Flight Plan\Mescalero LiDAR (1 of 8).kml
KML file saved: C:/Farid/Projects/20250310 - MAT PN1 TO1/UAS/00_PLANNING/Wingtra Flight Plan\Mescalero LiDAR (2 of 8).kml
KML file saved: C:/Farid/Projects/20250310 - MAT PN1 TO1/UAS/00_PLANNING/Wingtra Flight Plan\Mescalero LiDAR (3 of 8).kml
KML file saved: C:/Farid/Projects/20250310 - MAT PN1 TO1/UAS/00_PLANNING/Wingtra Flight Plan\Mescalero LiDAR (4 of 8).kml
KML file saved: C:/Farid/Projects/20250310 - MAT PN1 TO1/UAS/00_PLANNING/Wingtra Flight Plan\Mescalero LiDAR (5 of 8).kml
KML file saved: C:/Farid/Projects/20250310 - MAT PN1 TO1/UAS/00_PLANNING/Wingtra Flight Plan\Mescalero LiDAR (6 of 8).kml
KML file saved: C:/Farid/Projects/20250310 - MAT PN1 TO1/UAS/00_PLANNING/Wingtra Flight Plan\Mescalero LiDAR (7 of 8).kml
KML file saved: C:/Farid/Projects/20250310 - MAT PN1 TO1/UAS/00_PLANNING/Wingtra Flight Plan\Mescalero LiDAR (8 of 8).kml
